In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)

import sys
sys.path.append("../..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [3]:
def get_hospital(new_hospital, new_pharmacy):
    # 병원
    all_hospital = pd.read_excel('../../' + new_hospital)
    tmp = all_hospital.loc[all_hospital['주소'].str.contains('평택시')][['요양기관명','주소','종별코드명']].reset_index().drop(columns=['index']).rename(columns={'요양기관명':'이름'})

    tmp = all_hospital.loc[all_hospital['주소'].str.contains('평택시')][['요양기관명','주소','종별코드명']].reset_index().drop(columns=['index']).rename(columns={'요양기관명':'이름'})
    tmp['위도'] = pd.NA
    tmp['경도'] = pd.NA

    for i, address in enumerate(tmp['주소']):
        lng, lat = get_coordinates(address)
        tmp.at[i, '위도'] = lat
        tmp.at[i, '경도'] = lng

    clinic = tmp.loc[tmp['종별코드명'] == '의원'].drop(columns=['종별코드명'])
    oriental = tmp.loc[(tmp['종별코드명'] == '한의원') | (tmp['종별코드명'] == '한방병원')].drop(columns=['종별코드명'])
    dentistry = tmp.loc[tmp['종별코드명'] == '치과의원'].drop(columns=['종별코드명'])
    hospital = tmp.loc[tmp['종별코드명'] == '병원'].drop(columns=['종별코드명'])
    nursing = tmp.loc[tmp['종별코드명'] == '요양병원'].drop(columns=['종별코드명'])
    general = tmp.loc[tmp['종별코드명'] == '종합병원'].drop(columns=['종별코드명'])
    public = tmp.loc[tmp['종별코드명'].str.contains('보건')].drop(columns=['종별코드명'])


    # 약국
    all_pharmacy = pd.read_excel('../../' + new_pharmacy)
    pharmacy = all_pharmacy.loc[all_pharmacy['주소'].str.contains('평택시')][['요양기관명','주소']].reset_index().drop(columns=['index']).rename(columns={'요양기관명':'이름'})
    pharmacy['위도'] = pd.NA
    pharmacy['경도'] = pd.NA

    for i, address in enumerate(pharmacy['주소']):
        lng, lat = get_coordinates(address)
        pharmacy.at[i, '위도'] = lat
        pharmacy.at[i, '경도'] = lng

    return clinic, oriental, dentistry, hospital, nursing, general, public, pharmacy

In [4]:
new_hospital = 'new_data/의료시설/1.병원정보서비스 2024.6.xlsx'
new_pharmacy = 'new_data/의료시설/2.약국정보서비스 2024.6.xlsx'

clinic, oriental, dentistry, hospital, nursing, general, public, pharmacy = get_hospital(new_hospital, new_pharmacy)

In [5]:
clinic.to_csv('../../preprocessed/의료시설/의원.csv', index=False)
oriental.to_csv('../../preprocessed/의료시설/한의원.csv', index=False)
dentistry.to_csv('../../preprocessed/의료시설/치과.csv', index=False)
hospital.to_csv('../../preprocessed/의료시설/일반병원.csv', index=False)
nursing.to_csv('../../preprocessed/의료시설/요양병원.csv', index=False)
general.to_csv('../../preprocessed/의료시설/종합병원.csv', index=False)
public.to_csv('../../preprocessed/의료시설/보건병원.csv', index=False)
pharmacy.to_csv('../../preprocessed/의료시설/약국.csv', index=False)